In [1]:
from pathlib import Path

import pandas as pd
import numpy as np

from utils import ProcessData, LinearParameters, apply_activation


In [2]:
class NeuralNetwork():
    def __init__(self, config):
        """
        Parameters:
        -----------
        config : Dict
            config['lp_reg'] = 0,1,2
            config['nodes'] = List[int]
            config['bias'] = List[Boolean]
            config['activators'] = List[str]

        Returns:
        --------
        None
        """
        self.config = config
        self.lp_reg = config['lp_reg']
        self.nodes = config['nodes']
        self.bias = config['bias']
        self.activators = config['activators']
        self.L = len(config['nodes']) - 1

    def compute_cost(self, y, a, params, lambda_=0.0, eps=1e-8):
        """
        Parameters:
        -----------
        y: array_like
        a: array_like
        params: class[Parameters]
        lambda_: float
            Default: 0.0
        eps: float
            Default: 1e-8

        Returns:
        --------
        cost: float
        """
        n = y.shape[1]

        # Compute regularization term
        R = 0
        for param in params.values():
            R += np.sum(np.abs(param.w) ** self.lp_reg)
        R *= (lambda_ / (2 * n))

        # Compute unregularized cost
        J = (-1 / n) * (np.sum(y * np.log(a + eps)) +
                        np.sum((1 - y) * np.log(1 - a + eps)))

        cost = float(np.squeeze(J + R))

        return cost

    def forward_propagation(self, x, params):
        """
        Parameters:
        -----------
        x : array_like
        params : Dict[class[Parameters]]
            params[l].w = Weights
            params[l].bias = Boolean
            params[l].b = Bias

        Returns:
        --------
        cache = Dict[arrau_like]
            cache['a'] = a
            cache['dg'] = dg

        """
        # Initialize dictionaries
        a = {}
        dg = {}

        a[0], dg[0] = apply_activation(x, self.activators[0])

        for l in range(1, self.L + 1):
            z = params[l].forward(a[l - 1])
            a[l], dg[l] = apply_activation(z, self.activators[l])

        cache = {'a': a, 'dg': dg}
        return cache

    def backward_propagation(self, y, params, cache):
        """
        Parameters:
        -----------
        y : array_like
        params : Dict[class[Parameters]]
            params[l].w = Weights
            params[l].bias = Boolean
            params[l].b = Bias
        cache : Dict[array_like]
            cache['a'] : array_like
            cache['dg'] : array_like
        
        Returns:
        --------
        None
        """

        # Retrieve cache
        a = cache['a']
        dg = cache['dg']

        # Initialize differentials along the network
        delta = {}
        delta[self.L] = (a[self.L] - y) / y.shape[1]

        for l in reversed(range(1, self.L + 1)):
            delta[l - 1] = dg[l- 1] * params[l].backward(delta[l], a[l - 1])

    def update_parameters(self, params, learning_rate=0.01):
        """
        Parameters:
        -----------
        params : Dict[class[Parameters]]
            params[l].w = Weights
            params[l].bias = Boolean
            params[l].b = Bias
        learning_rate : float
            Default : 0.01

        Returns:
        --------
        None
        """
        for param in params.values():
            param.update(learning_rate)

    def fit(self, x, y, learning_rate=0.01, lambda_=0.0, num_iters=10000):
        """
        Parameters:
        -----------
        x : array_like
        y : array_like
        learning_rate : float
            Default : 0.1
        lambda_ : float
            Default : 0.0
        num_iters : int
            Default : 10000

        Returns:
        --------
        costs : List[floats]
        params : class[Parameters]
        """
        # Initialize parameters per layer
        params = {}
        for l in range(1, self.L + 1):
            params[l] = LinearParameters(
                (self.nodes[l], self.nodes[l - 1]), self.bias[l])

        costs = []
        for i in range(num_iters):
            cache = self.forward_propagation(x, params)
            cost = self.compute_cost(y, cache['a'][self.L], params, lambda_)
            costs.append(cost)
            self.backward_propagation(y, params, cache)
            self.update_parameters(params, learning_rate)

            if i % 1000 == 0:
                print(f'Cost after iteration {i}: {cost}')

        return costs, params

    def evaluate(self, x, params):
        """
        Parameters:
        -----------
        x : array_like
        params : class[Parameters]

        Returns:
        --------
        predictions : array_like
        """
        cache = self.forward_propagation(x, params)
        a = cache['a'][self.L]
        predictions = (~(a < 0.5)).astype(int)
        return predictions

    def accuracy(self, x, y, params):
        """
        Parameters:
        -----------
        x : array_like
        y : array_like
        params : class[Parameters]

        Returns:
        --------
        accuracy : float
        """
        predictions = self.evaluate(x, params)
        aux = np.abs(predictions - y)
        accuracy = 1 - np.sum(aux) / y.shape[1]

        return accuracy
        

In [3]:
csv = Path('data/housepricedata.csv')
df = pd.read_csv(csv)
dataset = df.values
x = dataset[:, 0:10]
y = dataset[:, 10].reshape(-1, 1)
data = ProcessData(x.T, y.T, 0.15, 0.15)


In [4]:
config = {
    'lp_reg' : 2,
    'nodes' : [10, 32, 8, 1],
    'bias' : [False, True, True, True],
    'activators' : ['linear', 'relu', 'relu', 'sigmoid']
}

config_1 = {
        'lp_reg' : 2,
        'nodes' : [10, 32, 32, 1],
        'bias' : [False, True, True, True],
        'activators' : ['linear', 'relu', 'relu', 'sigmoid']
    }

model = NeuralNetwork(config)
costs, params = model.fit(data.train['x'], data.train['y'], 0.05, 0.03)
dev_acc = model.accuracy(data.dev['x'], data.dev['y'], params)
print(f'The dev accuracy: {dev_acc}')
test_acc = model.accuracy(data.test['x'], data.test['y'], params)
print(f'The test accuracy: {test_acc}')

Cost after iteration 0: 0.693150024228287
Cost after iteration 1000: 0.6930056724734572
Cost after iteration 2000: 0.2323146873195952
Cost after iteration 3000: 0.20807794045187175
Cost after iteration 4000: 0.19673190158839213
Cost after iteration 5000: 0.18219621383209497
Cost after iteration 6000: 0.16708155305067965
Cost after iteration 7000: 0.15178306708809583
Cost after iteration 8000: 0.13803811502859992
Cost after iteration 9000: 0.12729526477250894
The dev accuracy: 0.904109589041096
The test accuracy: 0.8995433789954338


In [5]:
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import Model, Input
from keras.layers import Dense

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
train = {'x': x_train, 'y': y_train}
test = {'x': x_test, 'y': y_test}
mu = np.mean(train['x'], axis=0, keepdims=True)
var = np.var(train['x'], axis=0, keepdims=True)
eps = 1e-8
train['x'] = (train['x'] - mu) / np.sqrt(var + eps)
test['x'] = (test['x'] - mu) / np.sqrt(var + eps)

In [7]:
## Define network layout
input_layer = Input(shape=(10,))
hidden_layer_1 = Dense(
    32,
    activation='relu',
    kernel_initializer='he_normal',
    bias_initializer='zeros'
)(input_layer)
hidden_layer_2 = Dense(
    32,
    activation='relu',
    kernel_initializer='he_normal',
    bias_initializer='zeros'
)(hidden_layer_1)
output_layer = Dense(
    1,
    activation='sigmoid',
    kernel_initializer='he_normal',
    bias_initializer='zeros'
)(hidden_layer_2)
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

2022-06-17 16:06:15.928302: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10)]              0         
                                                                 
 dense (Dense)               (None, 32)                352       
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,441
Trainable params: 1,441
Non-trainable params: 0
_________________________________________________________________


In [8]:
## Compile the deisred model
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [9]:
## Train the model
hist = model.fit(
    train['x'],
    train['y'],
    batch_size=32,
    epochs=150,
    validation_split=0.2,
    verbose=0
)

In [10]:
## Evaluate the model
test_scores = model.evaluate(test['x'], test['y'], verbose=2)
print(f'Test Loss: {test_scores[0]}')
print(f'Test Accuracy: {test_scores[1]}')

10/10 - 0s - loss: 0.3627 - accuracy: 0.8904 - 36ms/epoch - 4ms/step
Test Loss: 0.3627116084098816
Test Accuracy: 0.8904109597206116
